In [15]:
import torch
from torch import nn
import math

# Two Tower Model

This is the idea I suggested in the meeting. In it you have two embeddings: one for the users and one for the nudge. Their models can be changed and adapted over time, as long as the embedding stays the same. The matches between users and nudges can then be ranked by distance, which can be biased to suit our needs. The best approach for learning is to do it in daily cycles, the content still changes throughout the day as user input changes and geolocation changes but the model itself does not.

## Towers

These towers can be any trainable model as shown in an example below:

In [16]:
class TowerModel(nn.Module):

    def __init__(self, **config):
        super(TowerModel, self).__init__()
        self.config = config

        self.model1 = nn.Sequential(nn.Linear(config['input_dim_1'], config['hidden_dim_1']),
                                    *[nn.Softmax(dim=-1), nn.Linear(config['hidden_dim_1'], config['hidden_dim_1'])]*config['hidden_layers_1'],
                                    nn.ReLU())
        self.model2 = nn.Sequential(nn.Linear(config['input_dim_2'], config['hidden_dim_2']),
                                    *[nn.Softmax(dim=-1), nn.Linear(config['hidden_dim_2'], config['hidden_dim_2'])]*config['hidden_layers_2'],
                                    nn.ReLU())

        self.readout = nn.Linear(config['hidden_dim_1']+config['hidden_dim_2'], config['output_dim'])

    def forward(self, input):
        """
        look at different user data using different models and then produce an embeddign with the combined inputs
        :param input:
        :return: embedding
        """
        out1 = self.model1(input[..., :self.config['input_dim_1']])
        out2 = self.model2(input[..., self.config['input_dim_1']:])

        return self.readout(torch.concat((out1, out2), dim=-1))

We can see later that the models chosen are arbitrary, provided `config['output_dim']` remains unchanged. Most typically one would use different DNN networks to embed different data pieces and then combine their outputs. The best example of this may be to embed metadata of the nudges and separately embed the text.

## Combined

Below is how the model would look when set-up in the Two-Tower model. As we can see, the model takes in batches of users and nudges and rates them against each other. These ratings can then be used to create a ranking of nudges for each user. (I strongly recommend that this ranking is injected with some less ``good'' nudges)

In [17]:
class TwoTowerModel(nn.Module):
    """
    Two tower model for offline ranking of nudges against users (both are reduced in complexity through embedding).
    """

    def __init__(self, **config):
        super(TwoTowerModel, self).__init__()

        self.user_tower = TowerModel(**config)
        self.nudge_tower = TowerModel(**config)

    def forward(self, **inputs) -> torch.tensor:
        user_embedding = self.user_tower(inputs['user_input'])
        nudge_embedding = self.nudge_tower(inputs['nudge_input'])
        return torch.matmul(user_embedding, nudge_embedding.T) # compute the distance

## Multi-Gate Mixture of Experts

For many tasks we may have different objectives we would seek to optimise. For example, we may be interested in maximising user interaction number and duration. These are different tasks, which may have different optimal solutions.

Heuristically, to target this, we may choose to pass the choices and information to different networks targeting these different objectives. These networks are known as experts.

In [18]:
def Expert(**config) -> nn.Module:
    embedding_layer = nn.Linear(config['input_dim'], config['exp_hidden_dim'])
    hidden_layer = nn.Linear(config['exp_hidden_dim'], config['exp_hidden_dim'])
    output_layer = nn.Linear(config['exp_hidden_dim'], config['output_dim'])
    return nn.Sequential(*[embedding_layer, nn.Sigmoid(),
                           *[hidden_layer, nn.ReLU()] * config['exp_layers'],
                           output_layer])


def Gate(**config) -> nn.Module:
    embedding_layer = nn.Linear(config['input_dim'], config['gate_hidden_dim'])
    return nn.Sequential(embedding_layer, nn.Sigmoid(),
                         nn.Linear(config['gate_hidden_dim'], config['exp_num']))

In [19]:
class MultiGateMixExperts(nn.Module):

    def __init__(self, **config):
        super(MultiGateMixExperts, self).__init__()

        self.input_dim = config['input_dim']

        self.gate1 = Gate(**config)
        self.gate2 = Gate(**config)

        self.experts = [Expert(**config) for _ in config['exp_num']]

        self.utility1 = TowerModel(**config)

    def forward(self, inp):
        # input should be concatenation of:
        # user id, distance vector in embedding, user and nudge data and geodata
        f = torch.tensor([expert(inp) for expert in self.experts])
        g1, g2 = self.gate1(inp), self.gate2(inp)
        return torch.matmul(g1, f), torch.matmul(g2, f)

![An overview of the model](https://miro.medium.com/v2/resize:fit:914/format:webp/1*w0MonzJA7LMGUO2_Hcsd9w.png)

## Loss

The most common form of loss for nudge recommendation models is Cross Entropy Loss. In this we penalise unchosen options to take into account that a full evaluation of the preferences is not possible. This problem is addressed in a different way by the next method.

Cross entropy loss takes into account the relative abundance of labels and then uses the options presented to the user to estimate the loss. Consider presenting 5 different nudges to a user.

In [20]:
class CustomCrossEntropyLoss:

    """
    The most commonly found strategy is called in-batch negative sampling: for a specific observation
    in a batch we consider every other observation in this batch as negative. This is because a full
    evaluation will not be possible.

    :param label_probs: precompute the relative abundance of each label for the weighting
    :return loss
    """

    def __init__(self, label_probs: torch.tensor):
        self.label_probs = label_probs

    def __call__(self, true_labels: torch.tensor, logits: torch.tensor, training: bool = False) -> torch.tensor:
        batch_size, nb_candidates = logits.shape

        if training:
            label_probs = torch.zeros(true_labels.shape)
            for label in true_labels:
                label_probs[true_labels == label] = self.label_probs[label] * torch.ones(true_labels.shape)
            logits -= torch.log(label_probs)

            true_labels = torch.range(0, batch_size)

        loss = nn.functional.cross_entropy(logits, true_labels)
        return torch.sum(loss)

## References
- https://dl.acm.org/doi/pdf/10.1145/3219819.3220007


# Swing Algorithm

This algorithm is designed for online running and on the Alibaba website 1688 it has had a 72% click-through rate.

The following scoring system is used for candidate generation. We use the score function below to rate user-nudge combinations.

$w_u = \frac{1}{(cnt+5)^{0.35}}$

This is the user weighting, where $c, n$ and $t$ are user-interaction counts.

$w_\text{pair} = w_u * w_\tilde{u}$

These are the pair weightings.

score$(i,j)= \sum_\text{pairs} \frac{w_\text{pair}}{1+\text{intersection}}$

This score function considers the intersection of both user interactions. It can easily be implemented as follows:

In [21]:
u2items = [[0, 17, 3, 24, 135], [0, 3, 53, 392, 24, 2]]         # u2items = array of users and their items
# u2items[i] = items user i clicked on
# u2items[j] = items user j clicked on

i2i = {}

for i in range(0, len(u2items)):
    wi = math.pow(len(u2items[i]) + 5, -0.35)
    for j in range(i + 1, len(u2items)):
        intersection = u2items[i] and u2items[j]   # intersection = items both user i and user j clicked on
        wj = wi * math.pow(len(u2items[j]) + 5, -0.35)  # wj = product-pair score
        for product_id in intersection:
            i2i[product_id] = i2i.get(product_id, 0.0) + wj / (1 + len(intersection))   # i2i is incrementally updated as we loop through users (we won't use a loop in production)

The ranking done afterwards is a much more sensitive task. We have traded off accuracy for speed in candidate generation. This final step is the most model dependent and we need to ensure fast computation. Common examples used, include gradient boosted trees and deep learning models such as Multi gate mixture of exports. Ranking can be framed as either a classification or learning to rank problem. As a classification problem, we can score candidates based on probability of click or purchase. Logistic regression with crossed features is simple to implement and a difficult baseline to beat. Decision trees are also commonly used. As a learning to rank problem, commonly used algorithms include LambdaMart, XGBoost, and LightGBM.

Online candidate generation is very intensive and often can be circumnavigated. The advantages of this approach test on its adaptability to user flows, but this can be accounted for with adaptive recommendation. In our case, with few nudges, we don’t need to be able to generate large collections of candidates from even larger samples. We can rely on generating a couple dozen candidates per user per day.

## References
- https://eugeneyan.com/writing/real-time-recommendations/